In [ ]:
%matplotlib inline


# Observer-based V/Hz controlled 2.2-kW PMSM drive

This example simulates observer-based V/Hz control of a 6.7-kW SyRM drive.
drive.


Import the package.



In [ ]:
import motulator as mt
import numpy as np

Compute base values based on the nominal values (just for figures).



In [ ]:
base = mt.BaseValues(
    U_nom=370, I_nom=15.5, f_nom=105.8, tau_nom=20.1, P_nom=6.7e3, p=2)

Configure the system model.



In [ ]:
motor = mt.SynchronousMotor(p=2, R_s=.54, L_d=41.5e-3, L_q=6.2e-3, psi_f=0)
mech = mt.Mechanics(J=.015)
conv = mt.Inverter()
mdl = mt.SynchronousMotorDrive(motor, mech, conv)

Configure the control system.



In [ ]:
pars = mt.SynchronousMotorVHzObsCtrlPars(
    p=2,
    R_s=.54,
    L_d=41.5e-3,
    L_q=6.2e-3,
    psi_f=0,
    psi_s_nom=np.sqrt(2/3)*base.U_nom/(2*np.pi*base.f_nom)
)
ctrl = mt.SynchronousMotorVHzObsCtrl(pars)

Set the speed reference and the external load torque.



In [ ]:
# Speed reference
times = np.array([0, .125, .25, .375, .5, .625, .75, .875, 1])*8
values = np.array([0, 0, 1, 1, 0, -1, -1, 0, 0])*base.w
ctrl.w_m_ref = mt.Sequence(times, values)
# External load torque
times = np.array([0, .125, .125, .875, .875, 1])*8
values = np.array([0, 0, 1, 1, 0, 0])*base.tau_nom
mdl.mech.tau_L_ext = mt.Sequence(times, values)

Create the simulation object and simulate it. You can also enable the PWM
model (which makes simulation slower). One-sampling-period computational
delay is modeled.



In [ ]:
sim = mt.Simulation(mdl, ctrl, pwm=False, delay=1)
sim.simulate(t_stop=8)

Plot results in per-unit values. By omitting the argument `base` you can plot
the results in SI units.



In [ ]:
mt.plot(sim, base=base)